In [ ]:
-- require 'torch'
-- require 'gnuplot'
-- require 'rnn'
-- require 'image'
-- Plot = require 'itorch.Plot'
-- -- scatter plots
-- plot = Plot():circle(torch.range(1, #err), 
--                 err, 'red', 'RMSE'):draw()
-- plot:line(torch.range(1, #err), 
--                 err, 'red', ''):redraw()
-- plot:title('Plot of Loss Function'):redraw()
-- plot:xaxis('Iterations'):yaxis('Mean Squared Error'):redraw()
-- plot:legend(true)
-- plot:redraw()

In [1]:
require 'rnn'

true	


In [165]:
setences = torch.LongTensor{{0, 1, 3, 4}, {2, 1, 4, 3}}:t()
query = torch.LongTensor{{0,0,3}, {1,3,2}}:t()
actions = torch.round(torch.rand(2, 1))

batch_size = 2
vocab_size = 4
embed_dim = 10
outputSize = 1

lstm1 = nn.Sequential()
lstm1:add(nn.LookupTableMaskZero(vocab_size, embed_dim)) -- will return a sequence-length x batch-size x embedDim tensor
lstm1:add(nn.SplitTable(1, embed_dim)) -- splits into a sequence-length table with batch-size x embedDim entries
lstm1:add(nn.Sequencer(nn.LSTM(embed_dim, embed_dim)))
lstm1:add(nn.SelectTable(-1)) -- selects last state of the LSTM
lstm1:add(nn.Linear(embed_dim, embed_dim)) -- map last state to a score for classification

lstm2 = nn.Sequential()
lstm2:add(nn.LookupTableMaskZero(vocab_size, embed_dim)) -- will return a sequence-length x batch-size x embedDim tensor
lstm2:add(nn.SplitTable(1, embed_dim)) -- splits into a sequence-length table with batch-size x embedDim entries
lstm2:add(nn.Sequencer(nn.LSTM(embed_dim, embed_dim)))
lstm2:add(nn.SelectTable(-1)) -- selects last state of the LSTM
lstm2:add(nn.Linear(embed_dim, embed_dim)) -- map last state to a score for classification

mlp = nn.Sequential()
mlp:add(nn.Linear(1, embed_dim))
mlp:add(nn.Linear(embed_dim, embed_dim))

p = nn.ParallelTable()
p:add(lstm1)
p:add(lstm2)
p:add(mlp)
-- p:add(nn.JoinTable(1))

In [166]:
preds = p:forward{setences, query, actions}

In [167]:
preds

{
  1 : DoubleTensor - size: 2x10
  2 : DoubleTensor - size: 2x10
  3 : DoubleTensor - size: 2x10
}


In [168]:
nn.JoinTable(1):forward(preds)

 0.2023 -0.0254  0.2167 -0.1812  0.1803  0.0503  0.0483  0.3013  0.1813  0.2048
 0.2138 -0.0696  0.2277 -0.1014  0.1545 -0.1174 -0.0189  0.2190  0.2132  0.2489
 0.0898  0.2730 -0.0784 -0.2054  0.1235  0.0971 -0.2758 -0.2466 -0.1786 -0.2513
 0.1065  0.2532 -0.1920 -0.2423  0.0342  0.1038 -0.1618 -0.3776 -0.1152 -0.2435
-0.0756 -0.3303  0.0474 -0.2289  0.9439 -0.1772  0.4493 -0.2737 -0.0069 -0.7997
-0.1611  0.5144 -0.6788 -0.1739  0.6920 -0.1723  0.3786 -0.4050 -0.3312 -0.4858
[torch.DoubleTensor of size 6x10]



In [170]:
x = torch.randn(2, 10)
y = torch.randn(2, 10)
z = torch.randn(2, 10)

tst = {x, y, z}

print(tst)
print(nn.JoinTable(1):forward(tst))

{
  1 : DoubleTensor - size: 2x10
  2 : DoubleTensor - size: 2x10
  3 : DoubleTensor - size: 2x10
}
-0.7041 -0.0438 -0.9827 -0.3147 -0.4290 -0.4894  1.4600 -0.4579 -0.6010  0.6675
-0.0038 -1.4663  0.2207  1.1462 -1.7009 -0.2944 -0.1020 -0.8489  1.6325  0.2674
-1.0698 -0.7443 -0.1849  0.7632 -1.1265 -0.1791 -0.7780  0.0638  0.1231 -1.7345
-0.7185 -0.7374  0.8541 -1.2832 -0.2002  1.3910  1.5279  0.3525  0.1520  1.5547
-0.9035  0.8848  1.0488  2.0511  0.8989 -0.0558  0.9162 -0.4268  1.0088 -2.2604
 0.3769  0.5136  0.4464 -0.4765 -1.8698  1.6459  1.7850  0.0352 -0.5310 -0.0497
[torch.DoubleTensor of size 6x10]

